In [1]:
import numpy as np
import os
import pandas as pd

work_dir = r'D:\Documents\course\junior\BigDataAna\12-20大作业'
dataset_dir = work_dir+r'\datasets'+'\\'
# print(dataset_dir)

# 获取数据集
train_data = pd.read_csv(dataset_dir+'train_set.csv')
train_data.info()
train_data.drop('timestamp', axis=1, inplace=True)
train_data.info()
train_data.tail()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99904 entries, 0 to 99903
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   userId     99904 non-null  int64  
 1   movieId    99904 non-null  int64  
 2   rating     99904 non-null  float64
 3   timestamp  99904 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.0 MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99904 entries, 0 to 99903
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   userId   99904 non-null  int64  
 1   movieId  99904 non-null  int64  
 2   rating   99904 non-null  float64
dtypes: float64(1), int64(2)
memory usage: 2.3 MB


,userId,movieId,rating
99899,671,6268,2.5
99900,671,6269,4.0
99901,671,6365,4.0
99902,671,6385,2.5
99903,671,6565,3.5


In [7]:
# 构建matrix
minihash = True
proj_func_num = 1000
mat = train_data.pivot_table(index=['userId'], columns=['movieId'], values=['rating'])
mat.fillna(0,inplace=True)
util_mat = np.array(mat.values)
movie_id2index = {mat.columns[i][1]:i for i in range(util_mat.shape[1])}
user_num = util_mat.shape[0]
movie_num = util_mat.shape[1]


In [8]:
# 构建sim mat
from time import time
start_time = time()
if minihash:
    mini_mat = np.array(util_mat)
    mini_mat[util_mat>=3.0] = 1
    mini_mat[util_mat<3.0] = 0
    # 构建哈希签名矩阵
    sig_mat = np.zeros((proj_func_num, user_num)) # 有个问题 用户对所有电影评分都小于3时，数组为全0， 没有第一个为1的位置,是否算其为相似
    for i in range(proj_func_num):
        pai = [j for j in range(movie_num)]
        np.random.shuffle(pai)
        pai_index = np.argsort(pai)
        for j in range(user_num):
            for index in pai_index:
                if mini_mat[j, pai[index]] == 1:
                    sig_mat[i, j] = pai[index]
                    break


    # 计算jiccard相似度
    sig_mat = sig_mat.T # 用行来算比较快
    sim_mat = np.eye(user_num)
    for i in range(user_num):
        for j in range(i+1, user_num):
            sim_mat[i,j] = len([sig_mat[i,k] == sig_mat[j, k] for k in range(proj_func_num)]) / proj_func_num


else:
    sim_mat = np.corrcoef(util_mat)
    sim_mat[sim_mat<0] = 0
end_time = time()
print("use time(s):", end_time-start_time)

use time(s): 151.75450205802917


In [9]:
K = 10
n = 5
def recommender(mode, userId, util_mat, sim_mat):
    userId = userId-1  # userID 从1开始，矩阵从0开始
    user_num = util_mat.shape[0]
    movie_num = util_mat.shape[1]
    if mode == 1: # 1为top k推荐 0为预测评分
        user_rate = np.zeros(util_mat.shape[1])
    else:
        user_rate = np.copy(util_mat[userId])


    sort_sim_index = np.argsort(sim_mat[userId])[::-1]
    for cur_movie in range(util_mat.shape[1]):
        if util_mat[userId, cur_movie] == 0:
            total = 0
            pred = 0
            rec_num = 0
            for another_user in sort_sim_index:
                another_user_rate = util_mat[another_user, cur_movie]
                if another_user_rate != 0 and rec_num < K:
                    pred += sim_mat[userId, another_user]*another_user_rate
                    total += sim_mat[userId, another_user]
                    rec_num += 1
                    # if cur_movie == 3303:
                    #     print('pred',pred)
                    #     print('total',total)
                    #     print('cur_movie:', cur_movie)
                    #     print('another user rate:', another_user_rate)
                    #     print('sim:', sim_mat[userId, another_user])
            # if cur_movie == 3303:
            #     print('pred',pred)
            #     print('total',total)
            #     print('rate', pred/total)
            #     print('cur_movie:', cur_movie)
            user_rate[cur_movie] = pred/total

    if mode == 0:
        return user_rate
    else:
        user_rate[np.isnan(user_rate)] = 0
        rec_index = np.argsort(user_rate)[-n:]
        # print(user_rate[rec_index])
        movie_id_list = list(mat.columns)
        rec = [movie_id_list[i][1] for i in rec_index]
        return rec




recommender(1, 600,util_mat, sim_mat)

D:\Programs\Anaconda\envs\pytorch\lib\site-packages\ipykernel_launcher.py:36: RuntimeWarning: invalid value encountered in double_scalars


[127728, 3129, 2252, 100553, 4190]

In [10]:
# 读取测试集
test = pd.read_csv(dataset_dir+'test_set.csv')
test.drop('timestamp', axis=1, inplace=True)
users, movies, ratings = test['userId'], test['movieId'], test['rating']
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   userId   100 non-null    int64  
 1   movieId  100 non-null    int64  
 2   rating   100 non-null    float64
dtypes: float64(1), int64(2)
memory usage: 2.5 KB


In [11]:
from time import time
# 开始预测
start_time = time()
preds = []
pred_cache = {}
for i in range(len(test)):
    print('%d/%d...' % (i+1, len(test)))
    if users[i] not in pred_cache.keys():
        rates = recommender(0, users[i], util_mat, sim_mat)
        preds.append(rates[movie_id2index[movies[i]]])
        pred_cache[users[i]] = rates
    else:
        rates = pred_cache[users[i]]
        preds.append(rates[movie_id2index[movies[i]]])

SSE = np.sum(np.square(preds - ratings))
end_time = time()
print("use:", end_time-start_time)

SSE

1/100...
2/100...
3/100...
4/100...
5/100...
6/100...
7/100...
8/100...
9/100...
10/100...
11/100...
12/100...
13/100...
14/100...
15/100...
16/100...
17/100...
18/100...
19/100...
20/100...
21/100...
22/100...
23/100...
24/100...
25/100...
26/100...
27/100...
28/100...
29/100...
30/100...
31/100...
32/100...
33/100...
34/100...
35/100...
36/100...
37/100...
38/100...
39/100...
40/100...
41/100...
42/100...
43/100...
44/100...
45/100...
46/100...
47/100...
48/100...
49/100...
50/100...
51/100...
52/100...
53/100...
54/100...
55/100...
56/100...
57/100...
58/100...
59/100...
60/100...
61/100...
62/100...
63/100...
64/100...
65/100...
66/100...
67/100...
68/100...
69/100...
70/100...
71/100...
72/100...
73/100...
74/100...
75/100...
76/100...
77/100...
78/100...
79/100...
80/100...
81/100...
82/100...
83/100...
84/100...
85/100...
86/100...
87/100...
88/100...
89/100...
90/100...
91/100...
92/100...
93/100...
94/100...
95/100...
96/100...
97/100...
98/100...
99/100...
100/100...
use: 171

D:\Programs\Anaconda\envs\pytorch\lib\site-packages\ipykernel_launcher.py:36: RuntimeWarning: invalid value encountered in double_scalars


90.69321180555556

In [2]:
import numpy as np
from scipy.spatial.distance import pdist,squareform
a = np.array([1,2,3,4,5,6,7,8,9,10])
for i in range(10):
    np.random.shuffle(a)
    print(a)

[ 2  5  1  7 10  8  3  6  9  4]
[ 7 10  3  9  6  8  2  5  1  4]
[ 6  5  3  2  9  4  7  1  8 10]
[ 7 10  8  3  6  4  2  1  9  5]
[ 1  9  4  2  7  3  5 10  8  6]
[ 1  9  4  6  2  8  3  5 10  7]
[ 5 10  8  2  6  7  1  3  4  9]
[ 3  9  7  8  5  2  4  6 10  1]
[ 9  4  5  8  6  3  1  2  7 10]
[ 2  5  3  9  4 10  8  7  1  6]
